This assumes that you already had .xml annotation files with one xml file per annotated frame given by rectlabel.  If you are using exported loopy annotations, you can use loppy_to_PASCAL.ipynb to convert the loopy annotations into the correct format.

In [1]:
import numpy as np
import os
import glob
import tensorflow as tf
from annotation_summary import get_annotation_summary
from create_label_map import create_label_map

from create_train_val import create_train_val as create_train_val_function
from create_tf_record import create_all_tf_records

In [2]:
should_train = True

In [5]:
#This is a folder with gazelle images and, within it, a folder called 'annotations' that has the .xml annotation files.
base_model_folder = '/home/golden/Dropbox/kenya-tracking/models/model-sept-1' 
base_data_folder = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/training-tfrecords/data-sept-1/'
root_annotations_folder = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/annotations/images-and-xmls/all-annotations' 

create_tf_records = True
create_labels = False
create_train_val = True

# annotations_folder = os.path.join(image_folder, 'annotations')
annotations_folders = glob.glob(root_annotations_folder + '/*')
annotations_folder = root_annotations_folder
print(annotations_folder)

# choose the 'pets' version of the model you are using here: 
# https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs
# one way to create the file is just copy and paste into a local text file
sample_config_file = '/home/golden/demo_model_training/faster_rcnn_resnet101_pets.config'


/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/annotations/images-and-xmls/all-annotations


In [6]:
class_counts = get_annotation_summary(annotations_folder)

There are 2454 .xml annotation files in /media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/annotations/images-and-xmls/all-annotations .
{'other': 1175, 'buffalo': 14064, 'zebra': 20693, 'gazelle': 5762, 'wbuck': 2830}
other 1175
{'other': 1175, 'buffalo': 14064, 'zebra': 20693, 'gazelle': 5762, 'wbuck': 2830}


In [7]:
#This is a list of the class names you want to use to clasiify the objects in the images.
#Generally they are the same as the classes that you annotated 
class_names = ['head']
#Comment out the bellow line to use the object names in the .xmls as class names
class_names = list(class_counts.keys())
print(class_names)

['other', 'buffalo', 'zebra', 'gazelle', 'wbuck']


In [8]:
#create nessisrary new folders

data_folder = os.path.join(base_data_folder, 'data')
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
model_folder = os.path.join(base_model_folder, 'model')
if not os.path.exists(model_folder):
    os.makedirs(model_folder)
    os.makedirs(os.path.join(model_folder, 'train'))
    os.makedirs(os.path.join(model_folder, 'eval'))


In [9]:
label_map_file = os.path.join(data_folder, 'label_map.pbtxt')

In [10]:
if create_labels:
    label_map_file = create_label_map(data_folder, class_names)

In [11]:
if create_train_val:
    create_train_val_function(annotations_folder)

/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/annotations/images-and-xmls/all-annotations/annotations/trainval.txt
/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/annotations/images-and-xmls/all-annotations


In [12]:
image_folder = root_annotations_folder

if create_tf_records:    
    fraction_of_examples_for_evaluation_set = .25

    create_all_tf_records(image_folder, label_map_file, data_folder,
                          fraction_of_examples_for_evaluation_set)

/home/golden/tensorflow/models/research/object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:


/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/annotations/images-and-xmls/all-annotations
sorting  2573 examples.


total: 2454
buffalo: 736
zebra: 1226
gazelle: 291
wbuck: 200
other: 1
552 training and 184 validation examples for buffalo.
919 training and 307 validation examples for zebra.
218 training and 73 validation examples for gazelle.
150 training and 50 validation examples for wbuck.
0 training and 1 validation examples for other.
1839 training and 615 validation examples.


# Now you have to do stuff

## First download the model you want from the tf model zoo:

Go to https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
and download the model you want. This tutorial assumes 'faster_rcnn_resnet101_coco' but choose as you want. Copy and paste the contents of the resulting folder (but not the actual folder) into the folder written in the ouput of the cell bellow.

In [11]:
print('copy the contents of the downloaded model folder into the following folder:')
print(model_folder)

copy the contents of the downloaded model folder into the following folder:
/home/golden/Dropbox/kenya-tracking/models/model-august/model


## Now you must edit the model .config file to match your use. 

Now you must edit the model pipeline.config file that you just downloaded and moved to match your use. The config file should be edited as described in the output of the following cell.

In [12]:
print('The .config file should be in this folder: ', model_folder)
print('The following edits should be made to the .config file:')
print()
print('Near the top of the file, the line that begins \'num_classes\' should be changed to...')
print('num_classes:', len(class_names))
print()
print('Beneath the line that says \'train_config: {batch_size: 1\' add the following line at the same indent as the above line (if running on gazelle head demo)')
print('batch_queue_capacity: 10')
print()
print('The line that begins \'fine_tune_checkpoint\' should be changed to...')
print('fine_tune_checkpoint: "' + os.path.join(model_folder, 'model.ckpt') + '"')
print()
print('Very important.  If you don\'t include the next line model will train forever.')
print('Beneath the line that says \'from_detection_checkpoint: true\' add the following line at the same indent as the above line (if running on gazelle head demo)')
print('num_steps: 5000')
print()
print('The line that begins \'label_map_path\' should be changed to...')
print('label_map_path: "' + os.path.join(data_folder, 'label_map.pbtxt') + '"')
print()
print('Within the braces ({}) labeled \'train_input_reader\' ')
print('The line that begins \'input_path\' should be changed to...')
print('input_path: "' + os.path.join(data_folder, 'train.record') + '"')
print()
print('The line that begins \'label_map_path\' should be changed to...')
print('label_map_path: "' + os.path.join(data_folder, 'label_map.pbtxt') + '"')
print()
print('Within the braces ({}) labeled \'eval_input_reader\' ')
print('The line that begins \'input_path\' should be changed to...')
print('input_path: "' + os.path.join(data_folder, 'val.record') + '"')
print()

The .config file should be in this folder:  /home/golden/Dropbox/kenya-tracking/models/model-august/model
The following edits should be made to the .config file:

Near the top of the file, the line that begins 'num_classes' should be changed to...
num_classes: 1

Beneath the line that says 'train_config: {batch_size: 1' add the following line at the same indent as the above line (if running on gazelle head demo)
batch_queue_capacity: 10

The line that begins 'fine_tune_checkpoint' should be changed to...
fine_tune_checkpoint: "/home/golden/Dropbox/kenya-tracking/models/model-august/model/model.ckpt"

Very important.  If you don't include the next line model will train forever.
Beneath the line that says 'from_detection_checkpoint: true' add the following line at the same indent as the above line (if running on gazelle head demo)
num_steps: 5000

The line that begins 'label_map_path' should be changed to...
label_map_path: "/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/tr

In [13]:
def run_training(train_py, eval_py, pipeline_config, train_dir, eval_dir, gpu_count=1, name=None):  
    print('running ', name)
    %env CUDA_VISIBLE_DEVICES=0
    train_process = subprocess.Popen(['python', train_py,
                            '--logtostderr', 
                            '--pipeline_config_path=' + pipeline_config,
                            '--train_dir=' + train_dir],
                            preexec_fn=os.setsid)
    if gpu_count > 1:

        time.sleep(60)

        %env CUDA_VISIBLE_DEVICES=1
        
        eval_process = subprocess.Popen(['python', eval_py,
                                '--logtostderr', 
                                '--pipeline_config_path=' + pipeline_config,
                                '--checkpoint_dir=' + train_dir,
                                '--eval_dir=' + eval_dir],
                                preexec_fn=os.setsid)

    train_process.wait()
    
    if gpu_count > 1:

        print('sleeping')

        time.sleep(180)

        print('shutting down eval')
        
        print('killing eval...')

        os.killpg(os.getpgid(eval_process.pid), signal.SIGTERM)
    
    time.sleep(30)

In [14]:
import subprocess
import signal
import time
import tensorflow as tf

#these must be changed to match where these files are saved on your computer
train_py = '/home/golden/tensorflow/models/research/object_detection/legacy/train.py'    
eval_py = '/home/golden/tensorflow/models/research/object_detection/legacy/eval.py'  

model_folder = os.path.join(base_model_folder, 'model')

pipeline_config = os.path.join(model_folder, 'pipeline.config')
train_dir = os.path.join(base_model_folder, 'train') 
eval_dir = os.path.join(base_model_folder, 'eval')
print(pipeline_config)

/home/golden/Dropbox/kenya-tracking/models/model-sept-1/model/pipeline.config


In [15]:
print('tensorboard --logdir="' + base_model_folder + '"')

tensorboard --logdir="/home/golden/Dropbox/kenya-tracking/models/model-sept-1"


In [16]:
if should_train:

    #only runs eval if two GPUs
    gpu_count = 2

    run_training(train_py, eval_py, pipeline_config, train_dir, eval_dir, gpu_count, name='')

running  
env: CUDA_VISIBLE_DEVICES=0
env: CUDA_VISIBLE_DEVICES=1
sleeping
shutting down eval
killing eval...


Now you are ready to train the model

In [1]:
import subprocess
def run_training(main_run_py, pipeline_path, model_dir, num_train_steps, num_eval_steps, save_checkpoint_steps):
    main_process = subprocess.Popen(['python', main_run_py, 
                                     '--pipeline_config_path='+ pipeline_path,
                                     '--model_dir=' + model_dir,
                                     '--num_train_steps=' + str(num_train_steps),
                                     '--num_eval_steps=' + str(num_eval_steps),
                                     '--save_checkpoint_steps=' + str(save_checkpoint_steps),
                                     '--alsologtostderr'])
    return main_process

In [2]:
import tensorflow as tf

main_run_py = '/home/golden/tensorflow/models/research/object_detection/model_main.py' 
pipeline_path = '/home/golden/Dropbox/kenya-tracking/models/model-august/model/pipeline.config' 
model_dir = '/home/golden/Dropbox/kenya-tracking/models/model-august'
num_train_steps = 500
num_eval_steps = 575
save_checkpoint_steps = 248

In [3]:
main_training = run_training(main_run_py, pipeline_path, model_dir, num_train_steps, num_eval_steps, save_checkpoint_steps)

In [4]:
import os
import signal
os.killpg(os.getpgid(main_training.pid), signal.SIGINT)

KeyboardInterrupt: 

tensorboard --logdir="/home/golden/Dropbox/kenya-tracking/models/model-august"


In [ ]:
if should_train:

    #only runs eval if two GPUs
    gpu_count = 2

    run_training(train_py, eval_py, pipeline_config, train_dir, eval_dir, gpu_count, name='')

running  
env: CUDA_VISIBLE_DEVICES=0


You can monitor training by running the following output in the terminal: 

# Export Graph

In [17]:
#these must be changed to match where these files are saved on your computer
export_py = '/home/golden/tensorflow/models/research/object_detection/export_inference_graph.py'
#choose step number to export (in train folder)
step_num = 150000
base_folder = base_model_folder 

subprocess.Popen(['python', export_py,
                '--input_type=image_tensor', 
                '--pipeline_config_path=' + pipeline_config,
                '--trained_checkpoint_prefix=' + os.path.join(train_dir, 'model.ckpt-' + str(step_num)),
                '--output_directory=' + os.path.join(base_folder, 'out_put_inference_graph.pb')],
                preexec_fn=os.setsid)



In [16]:
x = np.ones((1))

In [5]:
file = np.load('/media/golden/72FFC6EE48B5CF39/kenya-tracking/video-annotations/NOV08_2017_DJI_0117/boxes.npy')

In [4]:
print(file[0]['track'][0])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [6]:
file[0]

array([[ 0.66939765,  0.15950738,  0.71024215,  0.17530711],
       [ 0.27410191,  0.94650757,  0.311214  ,  0.96395314],
       [ 0.29083735,  0.32114401,  0.32803628,  0.33644333],
       [ 0.55746466,  0.25507566,  0.5864656 ,  0.27684891],
       [ 0.50881201,  0.64018017,  0.55308104,  0.65580165],
       [ 0.52180409,  0.21746561,  0.5460487 ,  0.23860645],
       [ 0.64702511,  0.5923444 ,  0.68180621,  0.611224  ],
       [ 0.67662221,  0.58801544,  0.72239596,  0.60465312],
       [ 0.32614514,  0.87391919,  0.35573795,  0.89649725],
       [ 0.28225872,  0.90321583,  0.3148874 ,  0.92367452],
       [ 0.6532827 ,  0.60080981,  0.69500142,  0.61800694],
       [ 0.3728126 ,  0.90018725,  0.40375561,  0.92283356],
       [ 0.26472819,  0.91882843,  0.29038703,  0.94201469],
       [ 0.49962825,  0.29934233,  0.53963041,  0.31307489]], dtype=float32)

In [2]:
import tensorflow as tf

In [10]:
graph1 = tf.Graph()
with graph1.as_default():
    x = tf.placeholder(tf.int32, name='placeholder')
    y = tf.multiply(x, tf.constant(10))
graph2 = tf.Graph()
with graph2.as_default():
    t = tf.constant(3, dtype=tf.int32)
with tf.Session(graph=graph1) as sess:
    output = sess.run(y, feed_dict={x:t})
    print(output)

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.For reference, the tensor object was Tensor("Const:0", shape=(), dtype=int32) which was passed to the feed with key Tensor("placeholder:0", dtype=int32).